In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from sklearn.metrics import roc_auc_score, roc_curve

import sys
sys.path.append('../src')
from models import DNMC, NMC, NSurv, MLP

In [2]:
TRAIN_SIZE = 500
VAL_SIZE = 100

x_train = np.random.rand(TRAIN_SIZE, 10)
y_idx = np.random.randint(50, size=(TRAIN_SIZE))
y_train = np.eye(50)[y_idx]
s_train = np.random.randint(2, size=(TRAIN_SIZE))

x_val = np.random.rand(VAL_SIZE, 10)
y_idx = np.random.randint(50, size=(VAL_SIZE))
y_val = np.eye(50)[y_idx]
s_val = np.random.randint(2, size=(VAL_SIZE))

In [3]:
def get_batches(*arrs, batch_size=1):
    l = len(arrs[0])
    for ndx in range(0, l, batch_size):
        yield (arr[ndx:min(ndx + batch_size, l)] for arr in arrs)

In [4]:
import time

def train_model(model, n_epochs, batch_size=50, learning_rate=1e-3):

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    #@tf.function
    def train_step(x, y, s):
        with tf.GradientTape() as tape:
            train_loss, train_nll = model.loss(x, y, s)
        grads = tape.gradient(train_loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        return train_loss, train_nll

    #@tf.function
    def test_step(x, y, s):
        val_loss, val_nll = model.loss(x, y, s)
        return val_loss, val_nll

    for epoch_idx in range(n_epochs):

        print("\nStart of epoch %d" % (epoch_idx,))
        start_time = time.time()

        batch_losses = []
        batch_nll = []

        for batch_idx, (xt, yt, st) in enumerate(get_batches(x_train, y_train, s_train, batch_size=batch_size)):

            train_loss, train_nll = train_step(xt, yt, st)

            batch_losses.append(train_loss)
            batch_nll.append(train_nll)

        # Display metrics at the end of each epoch.
        print('Epoch training loss: %.4f, NLL = %.4f' % (np.mean(batch_losses), np.mean(batch_nll)))

        batch_losses = []
        batch_nll = []

        # Run a validation loop at the end of each epoch.
        for batch_idx, (xv, yv, sv) in enumerate(get_batches(x_val, y_val, s_val, batch_size=batch_size)):

            val_loss, val_nll = test_step(xv, yv, sv)

            batch_losses.append(val_loss)
            batch_nll.append(val_nll)

        print('Validation training loss: %.4f, NLL = %.4f' % (np.mean(batch_losses), np.mean(batch_nll)))
        #print('Time taken: %.2fs' % (time.time() - start_time))

In [5]:
model = DNMC()
train_model(model, 8)


Start of epoch 0
Epoch training loss: 5.2067, NLL = 4.7728
Validation training loss: 5.2679, NLL = 4.8583

Start of epoch 1
Epoch training loss: 5.0190, NLL = 4.6280
Validation training loss: 5.1682, NLL = 4.7990

Start of epoch 2
Epoch training loss: 4.9091, NLL = 4.5563
Validation training loss: 5.0927, NLL = 4.7590

Start of epoch 3
Epoch training loss: 4.8159, NLL = 4.4963
Validation training loss: 5.0488, NLL = 4.7454

Start of epoch 4
Epoch training loss: 4.7415, NLL = 4.4501
Validation training loss: 5.0262, NLL = 4.7483

Start of epoch 5
Epoch training loss: 4.6804, NLL = 4.4123
Validation training loss: 5.0156, NLL = 4.7587

Start of epoch 6
Epoch training loss: 4.6274, NLL = 4.3785
Validation training loss: 5.0038, NLL = 4.7640

Start of epoch 7
Epoch training loss: 4.5801, NLL = 4.3468
Validation training loss: 5.0008, NLL = 4.7748


In [6]:
model = NMC()
train_model(model, 8)


Start of epoch 0
Epoch training loss: 4.8718, NLL = 4.8193
Validation training loss: 4.9568, NLL = 4.9051

Start of epoch 1
Epoch training loss: 4.7600, NLL = 4.7088
Validation training loss: 4.8923, NLL = 4.8416

Start of epoch 2
Epoch training loss: 4.6848, NLL = 4.6345
Validation training loss: 4.8503, NLL = 4.8005

Start of epoch 3
Epoch training loss: 4.6286, NLL = 4.5791
Validation training loss: 4.8230, NLL = 4.7740

Start of epoch 4
Epoch training loss: 4.5836, NLL = 4.5348
Validation training loss: 4.8047, NLL = 4.7563

Start of epoch 5
Epoch training loss: 4.5463, NLL = 4.4981
Validation training loss: 4.7934, NLL = 4.7456

Start of epoch 6
Epoch training loss: 4.5147, NLL = 4.4671
Validation training loss: 4.7883, NLL = 4.7409

Start of epoch 7
Epoch training loss: 4.4883, NLL = 4.4412
Validation training loss: 4.7878, NLL = 4.7409


In [7]:
model = NSurv()
train_model(model, 8)


Start of epoch 0
Epoch training loss: 4.8008, NLL = 4.7652
Validation training loss: 4.9237, NLL = 4.8886

Start of epoch 1
Epoch training loss: 4.7269, NLL = 4.6922
Validation training loss: 4.8711, NLL = 4.8369

Start of epoch 2
Epoch training loss: 4.6671, NLL = 4.6332
Validation training loss: 4.8253, NLL = 4.7918

Start of epoch 3
Epoch training loss: 4.6130, NLL = 4.5797
Validation training loss: 4.7850, NLL = 4.7520

Start of epoch 4
Epoch training loss: 4.5640, NLL = 4.5312
Validation training loss: 4.7529, NLL = 4.7202

Start of epoch 5
Epoch training loss: 4.5216, NLL = 4.4890
Validation training loss: 4.7311, NLL = 4.6987

Start of epoch 6
Epoch training loss: 4.4869, NLL = 4.4545
Validation training loss: 4.7207, NLL = 4.6884

Start of epoch 7
Epoch training loss: 4.4599, NLL = 4.4278
Validation training loss: 4.7199, NLL = 4.6878


In [8]:
model = MLP()
train_model(model, 8)


Start of epoch 0
Epoch training loss: 0.7148, NLL = 0.6968
Validation training loss: 0.7120, NLL = 0.6943

Start of epoch 1
Epoch training loss: 0.7105, NLL = 0.6931
Validation training loss: 0.7123, NLL = 0.6952

Start of epoch 2
Epoch training loss: 0.7098, NLL = 0.6931
Validation training loss: 0.7116, NLL = 0.6952

Start of epoch 3
Epoch training loss: 0.7086, NLL = 0.6924
Validation training loss: 0.7106, NLL = 0.6947

Start of epoch 4
Epoch training loss: 0.7074, NLL = 0.6916
Validation training loss: 0.7098, NLL = 0.6943

Start of epoch 5
Epoch training loss: 0.7062, NLL = 0.6910
Validation training loss: 0.7092, NLL = 0.6942

Start of epoch 6
Epoch training loss: 0.7053, NLL = 0.6905
Validation training loss: 0.7087, NLL = 0.6941

Start of epoch 7
Epoch training loss: 0.7045, NLL = 0.6900
Validation training loss: 0.7083, NLL = 0.6941


In [2]:
# ORIGINAL MODEL CODE HERE; IT HAS NOW BEEN MOVED TO src/models.py

class DNMC(Model):
    
    def __init__(self,
                 phi_layer_sizes=[100,], psi_layer_sizes=[100,], omega_layer_sizes=[100,],
                 e_layer_sizes=[100,], t_layer_sizes=[100,], c_layer_sizes=[100,],
                 importance_weights=[1., 1.],
                 include_censoring_density=True,
                 n_bins=50,
                 activation='relu',
                 ld=1e-3, lr=1e-3):
        
        super(DNMC, self).__init__()
        
        self.ld = ld
        self.lr = lr
        self.w0 = tf.convert_to_tensor(importance_weights[0], dtype=tf.float32)
        self.w1 = tf.convert_to_tensor(importance_weights[1], dtype=tf.float32)
        self.include_censoring_density = include_censoring_density
        self.n_bins = n_bins
        self.activation=activation
        
        self.phi_layers = [self.dense(ls) for ls in phi_layer_sizes]
        self.psi_layers = [self.dense(ls) for ls in psi_layer_sizes]
        self.omega_layers = [self.dense(ls) for ls in omega_layer_sizes]
        
        self.e_layers = [self.dense(ls) for ls in e_layer_sizes] + [Dense(1, activation='sigmoid')]
        self.t_layers = [self.dense(ls) for ls in t_layer_sizes] + [Dense(n_bins, activation='softmax')]
        self.c_layers = [self.dense(ls) for ls in c_layer_sizes] + [Dense(n_bins, activation='softmax')]
        
        self.phi_model = Sequential(self.phi_layers)
        self.psi_model = Sequential(self.psi_layers)
        self.omega_model = Sequential(self.omega_layers)
        
        self.e_model = Sequential(self.e_layers)
        self.t_model = Sequential(self.t_layers)
        self.c_model = Sequential(self.c_layers)
        
        
    def dense(self, layer_size):
        
        layer = Dense(
            layer_size,
            activation=self.activation,
            kernel_regularizer=regularizers.l2(self.lr),
            bias_regularizer=regularizers.l2(self.lr))
        
        return(layer)
        
    
    def forward_pass(self, x):
        
        self.phi = self.phi_model(x)
        self.psi = self.psi_model(x)
        self.omega = self.omega_model(x)
        
        self.e_pred = tf.squeeze(self.e_model(tf.concat([self.phi, self.psi], axis=-1)), axis=1)
        self.t_pred = self.t_model(tf.concat([self.psi, self.omega], axis=-1))
        self.c_pred = self.c_model(tf.concat([self.psi, self.omega], axis=-1))
        
        return self.e_pred, self.t_pred, self.c_pred
    
    
    def call(self, x):
        return self.forward_pass(x)
    
    
    def iweights(self, s):
        return tf.cast(s, dtype=tf.float32) * self.w1 + tf.cast((1 - s), dtype=tf.float32) * self.w0

    
    def loss(self, x, y, s):
        
        #assert np.ndim(s) == 1
        #assert np.ndim(y) == 2
        #assert np.shape(y)[1] == self.n_bins
        
        nll = tf.reduce_mean(self.iweights(s) * self.nll(x, y, s))
        l = nll + self.ld * tf.cast(self.mmd(x, s), dtype=tf.float32)
        l += tf.reduce_sum(self.losses)
        
        return l, nll
    
    
    def nll(self, x, y, s):
        
        #assert np.ndim(s) == 1
        #assert np.ndim(y) == 2
        #assert np.shape(y)[1] == self.n_bins
        
        yt = tf.cast(y, dtype=tf.float32)
        
        e_pred, t_pred, c_pred = self.forward_pass(x)
        
        ft = tf.reduce_sum(yt * t_pred, axis=1)
        Ft = tf.reduce_sum(yt * tf.math.cumsum(t_pred, axis=1), axis=1)
        fc = tf.reduce_sum(yt * c_pred, axis=1)
        Fc = tf.reduce_sum(yt * tf.math.cumsum(c_pred, axis=1), axis=1)
        
        l1 = e_pred * ft
        l2 = (1 - e_pred * (1 - Ft))
        
        if self.include_censoring_density:
            l1 = l1 * Fc
            l2 = l2 * fc
            
        ll = tf.cast(s, dtype=tf.float32) * tf.math.log(l1)
        ll += tf.cast((1 - s), dtype=tf.float32) * tf.math.log(l2)
        
        return -1 * ll
    
    
    def mmd(self, x, s, beta=1.):
        
        x0 = tf.boolean_mask(x, s == 0, axis=0)
        x1 = tf.boolean_mask(x, s == 1, axis=0)

        x0x0 = self._gaussian_kernel(x0, x0, beta)
        x0x1 = self._gaussian_kernel(x0, x1, beta)
        x1x1 = self._gaussian_kernel(x1, x1, beta)
        
        return tf.reduce_mean(x0x0) - 2. * tf.reduce_mean(x0x1) + tf.reduce_mean(x1x1)


    def _gaussian_kernel(self, x1, x2, beta=1.):
        return tf.exp(-1. * beta * tf.reduce_sum((x1[:, tf.newaxis, :] - x2[tf.newaxis, :, :]) ** 2, axis=-1))
